In [473]:
from splinter import Browser
from bs4 import BeautifulSoup
import pandas as pd
from selenium import webdriver
import string
import re

In [349]:
executable_path = {'executable_path': 'chromedriver.exe'}
browser = Browser('chrome', **executable_path, headless=False)

In [350]:
url = 'https://boardgamegeek.com/browse/boardgame'
browser.visit(url)

In [351]:
html = browser.html
soup = BeautifulSoup(html, 'html.parser')

In [380]:
table = soup.find("table", class_="collection_table")
rows = table.findAll('tr', id="row_")
len(rows)

100

In [396]:
# collecting basic stats from the top 100 table
game_list = []
for row in rows:
    
    thumbnail = row.find("td", class_="collection_thumbnail")
    img = thumbnail.img["src"]
    url = "https://boardgamegeek.com" + thumbnail.a["href"]
    name = thumbnail.a["href"].split("/")[3].replace("-", " ")
    name = string.capwords(name) #   the title() method capitalized 7th to 7Th
    rank = row.find("td", class_="collection_rank").a["name"]
    rating = row.findAll("td", class_="collection_bggrating")
    list_price = row.find("td", class_="collection_shop").div.text.split(" ")[0][12:18].strip()
    try: 
        lwst_amzn = row.find("td", class_="collection_shop").div.text.split(" ")[1][9:16].strip()
    except:
        lwst_amzn = None
        
    game_list.append({
        "name": name,
        "rank": rank,
        "ratings": {
            "geek_rating": rating[0].text.strip(),
            "avg_rating": rating[1].text.strip(),
            "num_voters": rating[2].text.strip()
                   },
        "MSRP": list_price,
        "lowest_amazon": lwst_amzn,
        "image": img,
        "url": url
    })
#     print(name)
#     print("-"*80)

# print(game_list)

In [563]:
# experimenting with regex
for game in game_list[0:1]:
    url = game["url"]
    browser.visit(url)
    html = browser.html
    soup = BeautifulSoup(html, 'lxml')
    header = soup.findAll("div", class_="game-header-title-info")[1]
    name = header.text.split("(")[0].strip()
#     name = re.findall(r'[A-Z][a-z]+', header)
    year = re.findall(r'\d{4}', header.text)[0]
    gameplay = soup.findAll("li", class_="gameplay-item")
    player_count = gameplay[0].span.text
    playing_time = gameplay[1].span.text
    weight = gameplay[3].findAll("span")[2].text.strip()
    credits = soup.findAll("div", class_="credits")[0].ul
    print(credits)

<ul> <li><div class="game-header-loading-bar col-xs-7"> </div></li> <li><div class="game-header-loading-bar col-xs-6"> </div></li> <li><div class="game-header-loading-bar col-xs-3"> </div></li> <li><div class="game-header-loading-bar col-xs-9"> </div></li> <li><div class="game-header-loading-bar col-xs-7"> </div></li> </ul>


In [391]:
games_df = pd.DataFrame(game_list)
# games_df.head()

In [393]:
url = 'https://boardgamegeek.com/browse/boardgame'
tables = pd.read_html(url)

In [394]:
df = tables[5]
df.columns = df.iloc[0]
df = df.drop([0])
df = df.drop(columns=['Shop'])
df = df.dropna(axis='columns')
df.head()

,Board Game Rank,Title,Geek Rating,Avg Rating,Num Voters
1,1,Gloomhaven (2017),8.614,8.92,22829
2,2,Pandemic Legacy: Season 1 (2015),8.492,8.65,29984
3,3,Through the Ages: A New Story of Civilization ...,8.270,8.55,14964
4,4,Terraforming Mars (2016),8.233,8.40,35939
5,5,Twilight Struggle (2005),8.179,8.33,33648


In [395]:
df["Publication Year"] = df["Title"].str.split("(", 1).str[1].str[0:4]
df["Title"] = df["Title"].str.split("(", 1).str[0]
# df.head()

In [387]:
# Regex Scrapping
# "IDENTIFIERS:

# \ used to escape a character
# \d any number
# \D anything but a number
# \s space
# \S anything but a space
# \w any character
# \W anything but a character
# . any character except a new line
# \. actually a period"
# \b whitespace around words

# MODIFIERS:
      
# {1,3} we're expecting 1-3
# + Match 1 or more
# ? Match 0 or 1
# * Match 0 or more
# $ match the end of a string
# ^ match the beginning of a string
# | matches either or e.g. \d{1-3}|\w{5-6}
# [] Match range or 'variance' e.g. [A-Za-z] or [1-5a-qA-Z]
# {x} expecting 'x' amount

# WHITE SPACE CHARACTERS:
# \n new line
# \s space
# \t tab
# \e escape (rare)
# \f form feed (rare)
# \r return
      
# DON'T FORGET!:
# . + * ? [ ] $ ^ ( ) { } \ |
 